In [2]:
# 구글 드라이브 연결

from google.colab import drive
import shutil
import os

drive.mount('/content/drive', force_remount=True)

project_path = '/content/drive/MyDrive/examples/4장/data'
os.makedirs(project_path, exist_ok=True)

# 현재 경로를 /content/drive로 설정
os.chdir(project_path)
print("현재 작업 디렉토리:", os.getcwd())
os.listdir(project_path)

Mounted at /content/drive
현재 작업 디렉토리: /content/drive/MyDrive/examples/4장/data


['6_access_log_example.txt']

In [3]:
import re
import pandas as pd

# Apache Access 로그 형식을 위한 정규 표현식
log_pattern = r'(\d+\.\d+\.\d+\.\d+) - - \[(.*?)\] "(.*?)" (\d{3}) (\d+)'

# Access 로그를 파싱하는 함수
def parse_apache_log(log_line):
    match = re.match(log_pattern, log_line)
    if match:
        return {
            'ip': match.group(1),
            'timestamp': match.group(2),
            'request': match.group(3),
            'status': match.group(4),
            'size': match.group(5)
        }
    return None

# Access 로그 파일을 읽는 함수
def read_log_file(file_path):
    parsed_logs = []
    with open(file_path, 'r') as log_file:
        for line in log_file:
            parsed_log = parse_apache_log(line)
            if parsed_log:
                parsed_logs.append(parsed_log)
    return pd.DataFrame(parsed_logs)

# 텍스트 패턴을 기반으로 비정상 요청을 탐지하는 함수
def is_malicious_request(request):
    # 간단한 악의적인 패턴 탐지 (SQL 인젝션, 경로 탐색 공격 등)
    patterns = [
        r"union.*select",         # SQL 인젝션
        r"\.\./",                 # 경로 탐색 공격
        r"select.*from",          # SQL 인젝션
        r"information_schema",    # SQL 인젝션
        r"/wp-login\.php",        # WordPress 로그인 페이지 크롤링
        r"/xmlrpc\.php",          # XML-RPC 공격
        r"(GET|POST) /admin",     # 관리자 페이지 접근 시도
        r"(GET|POST) /login",     # 로그인 페이지 접근 시도
    ]
    # 패턴 중 하나라도 매칭되면 비정상 요청으로 간주
    for pattern in patterns:
        if re.search(pattern, request.lower()):
            return True
    return False

# 상태 코드와 패턴에 따라 로그 분류
def classify_request(log_df):
    # 상태 코드에 따른 분류
    log_df['category'] = log_df['status'].apply(
        lambda status: '정상 요청' if status.startswith('2') else '비정상 요청'
    )

    # 비정상 요청 중에서도 악의적인 패턴 탐지
    log_df['malicious'] = log_df['request'].apply(is_malicious_request)

    # 악의적인 패턴이 발견되면 '악성 요청'으로 분류
    log_df.loc[log_df['malicious'] == True, 'category'] = '악성 요청'

    return log_df

# 결과 출력 함수
def display_summary(log_df):
    print("총 로그 수:", len(log_df))
    print(log_df['category'].value_counts())
    print("\n악성 요청 샘플:")
    print(log_df[log_df['category'] == '악성 요청'].head())

# 메인 함수
if __name__ == "__main__":
    # 로그 파일 읽기(로그파일을 좌측 상단에 있는 파일탭을 이용하여 업로드 후 테스트)
    log_df = read_log_file('6_access_log_example.txt')


    # 로그 분류
    log_df = classify_request(log_df)

    # 결과 출력
    display_summary(log_df)


총 로그 수: 50
category
정상 요청     41
비정상 요청     8
악성 요청      1
Name: count, dtype: int64

악성 요청 샘플:
               ip                   timestamp  \
44  203.0.113.195  01/Sep/2024:00:00:45 +0000   

                                             request status  size category  \
44  GET /search?q=select%20*%20from%20users HTTP/1.1    200  1100    악성 요청   

    malicious  
44       True  
